In [82]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import os
from scipy.optimize import minimize

# Task 1b

In [83]:
s = np.loadtxt("Solution_file.txt")

In [84]:
vertices = np.loadtxt("box.txt")

In [85]:
#Internal Caliberation marix
focal_length = 2960.37845
principal_points = (1841.68855, 1235.23369)
camera_matrix = np.array([[focal_length,0,principal_points[0]],
                         [0,focal_length,principal_points[1]],
                         [0,0,1]],dtype="double")

In [86]:
#Ray-Box intersection method
def intersection(Op_x,Op_y,Op_z,Dir_x,Dir_y,Dir_z):
    P_min_x = 0.0
    P_min_y = 0.0
    P_min_z = 0.0
    P_max_x = 0.165
    P_max_y = 0.063
    P_max_z = 0.093
    
    ray_min = (P_min_x - Op_x)/Dir_x
    ray_max = (P_max_x - Op_x)/Dir_x
    
    
    if(ray_min > ray_max):
        ray_min,ray_max = ray_max,ray_min
    
    ray_min_y = (P_min_y - Op_y)/Dir_y
    ray_max_y = (P_max_y - Op_y)/Dir_y
    
    if(ray_min_y > ray_max_y):
        ray_min_y,ray_max_y = ray_max_y,ray_min_y
        
    if((ray_min > ray_max_y) or (ray_min_y) > ray_max):
        return 0
    

    if(ray_min_y > ray_min):
        ray_min = ray_min_y
        
    if(ray_max_y < ray_max):
        ray_max = ray_max_y
        
    ray_min_z = (P_min_z - Op_z)/Dir_z
    ray_max_z = (P_max_z - Op_z)/Dir_z
    
    if(ray_min_z > ray_max_z):
        ray_min_z,ray_max_z = ray_max_z,ray_min_z
        
    if((ray_min > ray_max_z) or (ray_min_z > ray_max)):
        return 0
    
    if(ray_min_z > ray_min):
        ray_min = ray_min_z
        
    if(ray_max_z < ray_max):
        ray_max = ray_max_z
    
    if(ray_min < ray_max):
        return ray_min
    else:
        return ray_max

In [87]:
rootdir = "/home/aditya/Documents/Sem_3/TDCV/Project_1/data_task1/init_texture/"
images = ["DSC_9743.JPG","DSC_9744.JPG","DSC_9745.JPG","DSC_9746.JPG","DSC_9747.JPG","DSC_9748.JPG","DSC_9749.JPG","DSC_9750.JPG"]
i = 0
filtered_coordinates = []
desc_filtered = []
for i in range(len(images)):
    
    rotation_vector = s[i][0:3].reshape(3,1)
    translation_vector = s[i][3:6].reshape(3,1)
    
    rotation_matrix = np.zeros(shape=(3,3))
    cv2.Rodrigues(rotation_vector,rotation_matrix) #Compute rotation matrix
    Q = np.dot(camera_matrix,rotation_matrix)
    q = np.dot(camera_matrix,translation_vector)
    opt_centre = -np.dot(np.linalg.inv(Q),q) #Compute optical centre
    
    
    img = cv2.imread(os.path.join(rootdir + images[i])) #Read the images
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    kp, des = sift.detectAndCompute(gray, None)
    kp_img = cv2.drawKeypoints(gray,kp,img,flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    cv2.imwrite('sift_keypoints_' + str(9743 + i) + '.jpg',kp_img)
    
    count = 0
    Y = []
    descriptor = []
    dir = np.zeros(shape = (np.shape(des)[0],3))
    sum = np.zeros(shape=(3,1))
    for j in range(np.shape(des)[0]):
        dir[j] = np.dot(np.linalg.inv(Q),[kp[j].pt[0],kp[j].pt[1],1])
        val = intersection(opt_centre[0],opt_centre[1],opt_centre[2],dir[j][0],dir[j][1],dir[j][2])
        if(val != 0):
            prod = val * dir[j]
            prod = prod.reshape(3,1)
            sum = opt_centre + prod
            count += 1
            sum=np.ravel(sum)
            Y.append(sum)
            descriptor.append(des[j])
    
    for m in range(count):
        filtered_coordinates.append(Y[m]) #Append the 3D coordinates
        desc_filtered.append(descriptor[m]) #Append the respective descriptors

# Task 2

In [125]:
desc_filtered = np.array(desc_filtered)
img_1 = cv2.imread("/home/aditya/Documents/Sem_3/TDCV/Project_1/tracking/color_000006.JPG",cv2.IMREAD_COLOR)
sift_1 = cv2.SIFT_create()
kp_1,des_1 = sift_1.detectAndCompute(img_1,None)
bf = cv2.BFMatcher()
matches = bf.match(des_1,desc_filtered)

matches = sorted(matches, key = lambda x:x.distance)
matches = matches[:66] 

In [140]:
image_points = []
threed_points = []
for i in range(len(matches)):
    image_points.append(kp_1[matches[i].queryIdx].pt)
    threed_points.append(filtered_coordinates[matches[i].trainIdx])
image_points = np.array(image_points) 
threed_points = np.array(threed_points)
dist_coeff = None

#Apply PnPRansac
success,rotation_vec_1,translation_vec_1,inliers = cv2.solvePnPRansac(threed_points,image_points,camera_matrix,dist_coeff,iterationsCount=400,reprojectionError=1)

print("Rotation vector:\n{0}".format(rotation_vec_1))
print("Translation vector:\n{0}".format(translation_vec_1))


Rotation vector:
[[ 2.20681372]
 [-0.69803872]
 [ 0.26384952]]
Translation vector:
[[-0.08342262]
 [-0.03428778]
 [ 0.81656125]]


In [141]:

file = open("solution_task_2.txt",'w')

np.savetxt(file,rotation_vec_1.flatten(),newline="\t")
np.savetxt(file,translation_vec_1.flatten(),newline="\t")

file.close()

# Task 3


In [ ]:
def func(camera_mat,rot_mat,t_vec,img_point):
    return ()

In [156]:
#Obtain the SIFT keypoints of the subsequqent images
img_2 = cv2.imread("/home/aditya/Documents/Sem_3/TDCV/Project_1/tracking/color_000007.JPG",cv2.IMREAD_COLOR)
sift_2 = cv2.SIFT_create()
kp_2,ds_2 = sift_new.detectAndCompute(img_2,None)

bf = cv2.BFMatcher()
matches_2 = bf.match(ds_2,desc_filtered)

matches_2 = sorted(matches_2, key = lambda x:x.distance)
matches_2 = matches_2[:100]

three_dim_points = []
image_plane_points = []
for i in range(len(matches_2)):
    three_dim_points.append(filtered_coordinates[matches_2[i].trainIdx])
    image_plane_points.append(kp_2[matches_2[i].queryIdx].pt)
    
    

(100, 3)
(100, 2)
